In [12]:
import pandas as pd
import numpy as np
from scipy.stats import binom
import matplotlib.pyplot as plt
import utils
from utils import Plane, Handler
import improved_utils
import visualize
import pickle

### Trabajo previo - Contexto y relevamiento de información

- En base al relevamiento de información realizado, observamos que la cantidad de vuelos que llegan y salen de Aeroparque son 300. Suponiendo que salen y llegan la misma cantidad de aviones, eso nos da un total de 150 aviones por día. Dado que nuestra simulación contempla solamente 18 horas del día, la cantidad de aviones que llegan a AEP por hora esta dada por la cuenta $\frac{150}{24} = 6.25 \approx 6$. Por ende, la tasa de arribos $\lambda = 6/60 = 0.1$ 

- Suponiendo que no hay congestión, y el avión puede viajar a la velocidad máxima permitida en cada tramo, el tiempo que tarda es el siguiente: $(\frac{100 - 50}{300} + \frac{50 - 15}{250} + \frac{15 - 5}{200} + \frac{5 - 0}{150}) \cdot 60 \approx 23 \ minutos$

- El aeropuerto de Rosario se encuentra a un total de 154 millas náuticas de AEP (apróximadamente 286km), por ende, siguiendo con la misma idea que en el inciso anterios: $(\frac{154 - 100}{500} + \frac{100 - 50}{300} + \frac{50 - 15}{250} + \frac{15 - 5}{200} + \frac{5 - 0}{150}) \cdot 60 \approx 30 \ minutos$. Esto no es consistente con los horarios publicados por Aerolíneas (1 hora de viaje aproximadamente). Sin embargo, esto puede deberse a que en nuestro cálculo no estamos considerando variables como el tiempo de despegue y aterrizaje, tiempo de aceleración y desaceleración, congestión, etc. Lo mismo aplica para el inciso anterior; los tiempos y valores se volverán más realistas a medida que realicemos las simulaciones.

- La máxima cantidad de aterrizajes que podrían suceder entre las 6 am y la medianoche está dada por la siguiente cuenta: $\frac{18 \cdot 60}{4}= 270$. Entonces, la cantidad máxima de aterrizajes en 18 horas suponiendo la restricción mencionada anteriormente es 270.

### Elecciones y Supuestos del modelado

A continuación dejamos algunas consideraciones y supuestos que tuvimos en cuenta a la hora de modelar la llegada de aviones a AEP.

1. **Orden del loop (por minuto):**  
   a) Se actualizan posiciones de todos los aviones  
   b) Se samplea si aparece un avión en el horizonte  
   c) Se actualizan parámetros (velocidad, estado, fila, etc.)

2. **Velocidad inicial:**  
   Todo avión que aparece entra con **300 nudos** (intervalo [100, 50)).

3. **Separación de seguridad:**  
   Misma política tanto para aviones hacia AEP como en reposicionamiento.

4. **Velocidad mínima en reposicionamiento:**  
   Se define `MIN_DEVIATION_SPEED`. Si un avión debe bajar por debajo → **se desvía a MVD**.

5. **Cierre de AEP:**  
   Los aviones mantienen el comportamiento estándar hasta AEP. Si al arribar AEP está cerrado, el avión se desvía a MVD. El resto de aviones permanece en vuelo sin cambios.

### Ejercicio 1: Simulación de Montecarlo y Visualización

In [ ]:
# h = Handler(N_ITERS = 1, SAVE_HISTORY=True, LAMBDA=0.1)
# results, s_stats, planes = h.simulate()
# histories = [p for p in planes[0]["plane_history"]]
# visualize.visualize(histories,5, 18*60, AIRCRAFT_SVG_PATH="plane.svg")

Exited visualization..


### Ejercicio 2: Simulación con promedio de 1 arribo por hora
Si el promedio de arribos es 1 avión por hora y nosotros medimos minuto a minuto, $\lambda=\frac{1}{60}$

In [14]:
ej2 = Handler(N_ITERS = 20_000, LAMBDA = 1/60, SAVE_HISTORY=True)
results, simulation_stats, plane_history = ej2.simulate()

Simulation::   0%|          | 0/20000 [00:00<?, ?it/s]

Simulation:: 100%|██████████| 20000/20000 [02:51<00:00, 116.30it/s]


In [15]:
exercise_2 = {
    "numeric_results": pd.DataFrame(results),
    "simulation_stats": pd.DataFrame(simulation_stats),
    "plane_history": plane_history  
}
with open("exercise_2.pkl", "wb") as f:
    pickle.dump(exercise_2, f)


### Ejercicio 3: Probabilidad de que lleguen 5 aviones en una hora

In [16]:
N_ITERS = 20_000
LAMBDA = 1/60
props_five = np.zeros(N_ITERS)
for i, sim in enumerate(plane_history):
    first_appearances = [sim["plane_history"][i][0][0] for i in range(len(sim["plane_history"]))]
    groups = np.zeros((18,))
    more_than_five = 0
    for t0 in first_appearances:
        min_to_hour = int(t0 // 60)
        groups[min_to_hour] += 1
    more_than_five += (groups >= 5).astype(int).sum() / 18
    props_five[i] = more_than_five

print("Proba de que lleguen más de 5 aviones en una hora:", props_five.mean())
print("Error de estimación:", 1.96 * props_five.std()/np.sqrt(N_ITERS))

Proba de que lleguen más de 5 aviones en una hora: 0.0032611111111111113
Error de estimación: 0.00018726722064577947


Podríamos calcularlo de forma analítica de la siguiente manera:

$$
\begin{aligned}
    \text{Sea } X &= 
    \begin{cases}
        1 & \text{si arribó un avión en ese minuto} \\
        0 & \text{si no}
    \end{cases} \\[6pt]
    &X \sim \text{Be}\!\left(\tfrac{1}{60}\right) \\[6pt]
    \text{Definimos } S &= \sum_{i=1}^{60} X_i \sim Bin(60, \frac{1}{60}) \\[6pt]
    \text{Luego} \ Y &= "\text{proba de que lleguen 5 aviones en una hora}" \\ &= \ P(S \ge 5) \\[6pt]
    \text{Así, podemos calcular analíticamente: } P(S \ge 5) &= 1 - P(S \lt 5)
\end{aligned}
$$

In [7]:
rv = binom(60, 1/60)
1 - rv.cdf(4)

np.float64(0.0032833719684949303)

Así, vemos que el valor teórico de $P(S \ge 5)$  es 0.00328 $\approx$ 0.0033, lo cual no está lejos del valor estimado

### Ejercicio 4: Simulación de arribos para distintas tasas de arribo

In [8]:
lambdas = [0.02, 0.1 , 0.2, 0.5, 1]
exercise_4 = []

for l in lambdas:
    h = Handler(N_ITERS=20_000, LAMBDA= l)
    res, sim_stats, _ = h.simulate()
    exercise_4.append({
        "lambda": l,
        "numeric_results": pd.DataFrame(res),
        "simulation_stats": pd.DataFrame(sim_stats)
    })
    del res, sim_stats, h

Simulation:: 100%|██████████| 20000/20000 [38:26<00:00,  8.67it/s] 


In [9]:
with open("exercise_4_results.pkl", "wb") as f:
    pickle.dump(exercise_4, f)

### Ejercicio 5: Simulación con interrupción de aterrizaje

In [10]:
lambdas = [0.02, 0.1 , 0.2, 0.5, 1]
results = []
exercise_5 = []
bounce_prob = 0.1

N_ITERS = 20_000
for l in lambdas:
    h = Handler(N_ITERS=N_ITERS, LAMBDA= l, PROP_BOUNCE=bounce_prob)
    res, sim_stats, _ = h.simulate()
    exercise_5.append({
        "lambda": l,
        "numeric_results": pd.DataFrame(res),
        "simulation_stats": pd.DataFrame(sim_stats)
    })
    del res, sim_stats, h

Simulation:: 100%|██████████| 20000/20000 [39:54<00:00,  8.35it/s] 


In [11]:
with open("exercise_5_results.pkl", "wb") as f:
    pickle.dump(exercise_5, f)

In [ ]:
# VISUALIZACION CON REBOTE
# h = Handler(N_ITERS = 1, LAMBDA=0.1, PROP_BOUNCE=bounce_prob, SAVE_HISTORY=True)
# results, s_stats, planes = h.simulate()
# histories = [p for p in planes[0]["plane_history"]]
# visualize.visualize(histories,2.5, 18*60, AIRCRAFT_SVG_PATH="plane.svg")

Simulation:: 100%|██████████| 1/1 [00:00<00:00, 11.46it/s]


Exited visualization..


### Ejercicio 6: Simulación con cierre de media hora

In [ ]:
lambdas = [0.02, 0.1 , 0.2, 0.5, 1]
exercise_6 = []
N_ITERS = 20_000
for l in lambdas:
    h = Handler(N_ITERS=N_ITERS, LAMBDA= l, CLOSING_TIME=True)
    res, sim_stats, _ = h.simulate()
    exercise_6.append({
        "lambda": l,
        "numeric_results": pd.DataFrame(res),
        "simulation_stats": pd.DataFrame(sim_stats)
    })
    del res, sim_stats, h

Simulation:: 100%|██████████| 1/1 [00:00<00:00,  7.94it/s]


In [ ]:
# VISUALIZACION CON CLOSING_TIME
# h = Handler(N_ITERS = 1, LAMBDA=0.1, CLOSING_TIME=True, SAVE_HISTORY=True)
# results, s_stats, planes = h.simulate()
# histories = [p for p in planes[0]["plane_history"]]
# visualize.visualize(histories, 2.5, 18*60, AIRCRAFT_SVG_PATH="plane.svg")

Simulation:: 100%|██████████| 1/1 [00:00<00:00, 25.53it/s]


Exited visualization..


In [14]:
with open("exercise_6_results.pkl", "wb") as f:
    pickle.dump(exercise_6, f)

### Ejercicio 7: Cambios para mejorar el funcionamiento del aeropuerto

Para la siguiente experimentación y cambios de políticas vamos a usar $\lambda = 0.1$ que representa el valor que estimamos en función de los datos empíricos obtenidos de aerolíneas y otros portales de aviación.

In [15]:
LAMBDA = 0.1
N_ITERS = 20_000

new_speeds = Handler( LAMBDA=LAMBDA, N_ITERS=N_ITERS)
new_speeds_res, new_speeds_st, _ = new_speeds.simulate()

baseline = {
    "lambda": 0.1,
    "numeric_results": new_speeds_res,
    "simulation_stats": new_speeds_st
}

Simulation:: 100%|██████████| 20000/20000 [06:13<00:00, 53.49it/s]


In [16]:
with open("exercise_7_baseline.pkl", "wb") as f:
    pickle.dump(baseline, f)

In [17]:
# RELAJAMOS EL BUFFER EN 1 MINUTO (4 --> 3 y 5 --> 4)
MIN_THRESHOLD = 3
MIN_BUF = 4      

LAMBDA = 0.1
N_ITERS = 20_000

new_speeds = Handler(MIN_THRESHOLD=MIN_THRESHOLD,MIN_BUF=MIN_BUF, 
                     LAMBDA=LAMBDA, N_ITERS=N_ITERS)
new_speeds_res, new_speeds_st, _ = new_speeds.simulate()

changed_buffers = {
    "lambda": 0.1,
    "numeric_results": new_speeds_res,
    "simulation_stats": new_speeds_st
}

Simulation:: 100%|██████████| 20000/20000 [05:57<00:00, 56.00it/s]


In [18]:
with open("exercise_7_new_buffers.pkl", "wb") as f:
    pickle.dump(changed_buffers, f)

In [19]:
# RELAJAMOS EL AJUSTE DE VELOCIDAD: (NEXT_SPEED - 20) --> (NEXT_SPEED - 10)
SPEED_ADJUSTMENT = 10

LAMBDA = 0.1
N_ITERS = 20_000

new_speeds = Handler(SPEED_ADJUSTMENT=SPEED_ADJUSTMENT, 
                                    LAMBDA=LAMBDA, N_ITERS=N_ITERS,)

new_speeds_res, new_speeds_st, _ = new_speeds.simulate()

changed_speed_adjust = {
    "lambda": 0.1,
    "numeric_results": new_speeds_res,
    "simulation_stats": new_speeds_st
}

Simulation:: 100%|██████████| 20000/20000 [06:20<00:00, 52.62it/s]


In [20]:
with open("exercise_7_new_speed_adjust.pkl", "wb") as f:
    pickle.dump(changed_speed_adjust, f)

In [21]:
LAMBDA = 0.1
N_ITERS = 20_000

MIN_THRESHOLD = 5 # Ahora medido en MN
MIN_BUF = 10 # Ahora medido en MN tambien

new_speeds = improved_utils.Handler(MIN_THRESHOLD=MIN_THRESHOLD, MIN_BUF=MIN_BUF, 
                                    LAMBDA=LAMBDA, N_ITERS=N_ITERS,)

new_speeds_res, new_speeds_st, _ = new_speeds.simulate()

new_buffer_measurements = {
    "lambda": 0.1,
    "numeric_results": new_speeds_res,
    "simulation_stats": new_speeds_st
}

Simulation:: 100%|██████████| 20000/20000 [05:13<00:00, 63.85it/s]


In [22]:
with open("exercise_7_new_buffer_measurements.pkl", "wb") as f:
    pickle.dump(new_buffer_measurements, f)

In [ ]:
# h = improved_utils.Handler(N_ITERS = 1, LAMBDA=0.1, SAVE_HISTORY=True)
# results, s_stats, planes = h.simulate()
# histories = [p for p in planes[0]["plane_history"]]
# visualize.visualize(histories,2.5, 18*60, AIRCRAFT_SVG_PATH="plane.svg")

Simulation:: 100%|██████████| 1/1 [00:00<00:00, 14.59it/s]


Exited visualization..


In [24]:
# RELAJAMOS EL BUFFER EN 1 MINUTO (4 --> 3 y 5 --> 4)
MIN_THRESHOLD = 3
MIN_BUF = 4    
SPEED_ADJUSTMENT = 10    

LAMBDA = 0.1
N_ITERS = 20_000

new_speeds = Handler(MIN_THRESHOLD=MIN_THRESHOLD,MIN_BUF=MIN_BUF,
                     SPEED_ADJUSTMENT=SPEED_ADJUSTMENT, 
                     LAMBDA=LAMBDA, N_ITERS=N_ITERS)
new_speeds_res, new_speeds_st, _ = new_speeds.simulate()

all_combined = {
    "lambda": 0.1,
    "numeric_results": new_speeds_res,
    "simulation_stats": new_speeds_st
}

Simulation:: 100%|██████████| 20000/20000 [05:47<00:00, 57.57it/s]


In [25]:
with open("exercise_7_all_combined.pkl", "wb") as f:
    pickle.dump(all_combined, f)